# Diffusive toggle switch

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
from pysb.integrate import odesolve
from scripts.models.enzymatic_futile_cycle_pysb import model

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

plt.style.use("./scripts/output/notebooks/custom_style.mplstyle")
%matplotlib inline

In [ ]:
concentrations_ode = odesolve(model, np.arange(1.0))
slice_vec = []
for o in model.observables:
    slice_vec.append(int(np.round(concentrations_ode[o.name][-1])))
slice_vec = np.array(slice_vec)

## Load initial data

Exact FSP solution

In [ ]:
_ = np.load("scripts/reference_solutions/efc_ode_ref.npy")
exact_marginal = np.load("scripts/reference_solutions/efc_ode_ref.npy")

In [ ]:
print(exact_marginal[-1])

Matrix integrator

In [ ]:
with xr.open_dataset("output/efc_matrix/output_t10000.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    slice_vec = np.array(np.ones(6))
    idx_2D = np.array([0, 0])
    matrix_marginal = lr_sol.marginalDistributions()

### TTN integrator

#### Matrix

In [ ]:
tree_r20 = readTree("output/efc_r20_i_tau1e-4/output_t100000.nc")
_, ttn_marginal_r_20 = tree_r20.calculateObservables(np.zeros(6, dtype="int"))
tree_r25 = readTree("output/efc_r25_i_tau1e-3/output_t10000.nc")
_, ttn_marginal_r_25 = tree_r25.calculateObservables(np.zeros(6, dtype="int"))
tree_r30 = readTree("output/efc_r30_i_tau1e-3/output_t10000.nc")
_, ttn_marginal_r_30 = tree_r30.calculateObservables(np.zeros(6, dtype="int"))
tree_r35 = readTree("output/efc_r35_i_tau1e-3/output_t10000.nc")
_, ttn_marginal_r_35 = tree_r35.calculateObservables(np.zeros(6, dtype="int"))
tree_r40 = readTree("output/efc_r40_i_tau2e-3/output_t5000.nc")
_, ttn_marginal_r_40 = tree_r40.calculateObservables(np.zeros(6, dtype="int"))

# Tree

In [ ]:
tree_r30_10 = readTree("output/efc_r30-10_i_tau1e-3_h/output_t10000.nc")
_, ttn_marginal_r_30_10 = tree_r30_10.calculateObservables(np.zeros(6, dtype="int"))

### SSA

In [ ]:
idx_2D = np.array([0, 1])

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/enzymatic_futile_cycle_ssa_1e+05.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
ssa_marginal_1e5, _, _, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)
ssa_wall_time = 136.4

In [ ]:
ssa_1e6 = np.load("scripts/reference_solutions/enzymatic_futile_cycle_ssa_1e+06.npy")
ssa_1e6_sol = SSASol(ssa_1e6)
ssa_marginal_1e6, _, _, _ = ssa_1e6_sol.calculateObservables(slice_vec, idx_2D)
ssa_wall_time = 1189.3

## Convergence with increasing rank

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,8))
ax1.plot(np.arange(tree_r20.grid.n[0]), ttn_marginal_r_20[0], "v", label="TTN, $r=20$")
ax1.plot(np.arange(tree_r25.grid.n[0]), ttn_marginal_r_25[0], "<", label="TTN, $r=25$")
ax1.plot(np.arange(tree_r30.grid.n[0]), ttn_marginal_r_30[0], "^", label="TTN, $r=30$")
ax1.plot(np.arange(tree_r35.grid.n[0]), ttn_marginal_r_35[0], ">", label="TTN, $r=35$")
ax1.plot(np.arange(tree_r40.grid.n[0]), ttn_marginal_r_40[0], ">", label="TTN, $r=40$")
ax1.plot(np.arange(ssa_1e6_sol.n[0])+ssa_1e6_sol.n_min[0], ssa_marginal_1e6[-1][0], "k.", label="SSA, $10^6$ runs")
ax1.set_xlabel("$x_0$")

ax2.plot(np.arange(tree_r20.grid.n[3]), ttn_marginal_r_20[3], "v", label="TTN, $r=20$")
ax2.plot(np.arange(tree_r25.grid.n[3]), ttn_marginal_r_25[3], "<", label="TTN, $r=25$")
ax2.plot(np.arange(tree_r30.grid.n[3]), ttn_marginal_r_30[3], "^", label="TTN, $r=30$")
ax2.plot(np.arange(tree_r35.grid.n[3]), ttn_marginal_r_35[3], ">", label="TTN, $r=35$")
ax2.plot(np.arange(tree_r40.grid.n[3]), ttn_marginal_r_40[3], ">", label="TTN, $r=40$")
ax2.plot(np.arange(ssa_1e6_sol.n[1])+ssa_1e6_sol.n_min[1], ssa_marginal_1e6[-1][1], "k.", label="SSA, $10^6$ runs")
ax2.set_xlabel("$x_1$")

ax2.legend()
plt.savefig("plots/efc_fig1.pdf")

In [ ]:
time_series = TimeSeries("output/efc_r5_e_tau1e-5")
mass_err_r5 = time_series.getMassErr()

In [ ]:
time = time_series.time

In [ ]:
fig, ax = plt.subplots()
ax.plot(time, np.abs(mass_err_r5), label="TTN, $r=5$")
ax.set_xlabel("$t$")
ax.set_ylabel("$|\Delta m(t)|$")
ax.set_yscale("log")
ax.legend()
plt.savefig("plots/efc_fig2.pdf")